# Legge di Boyle e sua linearizzazione

*Sulla base della lettura di un file CSV, calcola i coefficienti della retta di regressione, le incertezze dei parametri e il coefficiente di correlazione utilizzando la funzione `linregress` del pacchetto SciPy.
Visualizza i risultati in tre grafici: il primo comprendente i dati e la retta di regressione, il secondo l'iperbole che ne deriva e nel terzo sono rappresentati i residui e la relativa deviazione standard.*

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress

In [ ]:
nome_file = input("Inserire il nome del file CSV: ")
file_in = open(nome_file, "r")
coppie_dati = np.loadtxt(file_in, delimiter = ",", comments = '#', usecols = (0,1))

Viene utilizzata la funzione `linregress()` della libreria SciPy. Riportati i risultati nella tuple `esiti` dalla quale vengono estratti gli elementi `slope` e `intercept`.
La variabile `dati_x` rappresenta il reciproco della pressione.

In [ ]:
n = len(coppie_dati)

nparrayX_Y = coppie_dati.transpose()
dati_y = nparrayX_Y[0]
dati_pressione = nparrayX_Y[1]
dati_x = 1/dati_pressione
file_in.close()

esiti = linregress(dati_x, dati_y)

a = esiti.slope
b = esiti.intercept

Qui si estraggono le stime delle deviazioni standard e il coefficiente di correlazione.

In [ ]:
previsioni_y = a*dati_x + b
residui = dati_y - previsioni_y
massimo_residui = max(abs(residui))

sigma = np.std(residui, ddof=2)
sigma_a = esiti.stderr
sigma_b = esiti.intercept_stderr
coeff_correlazione = esiti.rvalue

In [ ]:
min_x = min(dati_x)
max_x = max(dati_x)
delta_x = (max_x-min_x)/15

min_pres = min(dati_pressione)
max_pres = max(dati_pressione)
delta_pres = (max_pres-min_pres)/15

Si costruisce la retta di regressione e quelle che delimitano la fascia di incertezza. 

In [ ]:
x = np.linspace(min_x-delta_x, max_x+delta_x, 100)
y = a*x + b
y_sup = (a+sigma_a)*x + (b+sigma_b)
y_inf = (a-sigma_a)*x + (b-sigma_b)

Qui vengono definite le corrispondenti iperboli

In [ ]:
x_pres = np.linspace(min_pres-delta_pres, max_pres+delta_pres, 100)
y_pres = a/x_pres + b
y_pressup = (a+sigma_a)/x_pres + (b+sigma_b)
y_presinf = (a-sigma_a)/x_pres + (b-sigma_b)

Primo grafico: dispersione dei dati, retta di regressione e fascia di incertezza. In ascissa è posto il reciproco della pressione cioè $z=1/P$.

In [ ]:
plt.grid(which = 'both', color = '.85', linestyle = '-', linewidth=1)
plt.vlines(dati_x, dati_y-sigma, dati_y+sigma, linewidth = .5, color = 'b')
plt.fill_between(x, y_inf, y_sup, alpha = .1, linewidth = 0, color='r')
plt.plot(x, y, color = 'red', linewidth = 2, label = 'retta di regressione')
plt.scatter(dati_x, dati_y, s = 20, c = 'cornflowerblue', zorder = 3, label = 'dati rilevati')
plt.title("Legge di Boyle (linearizzata): modello $y=ax+b$")
plt.xlabel('z=1/P (kPa$^{-1}$) ')
plt.ylabel('volume (cm$^3$)')
plt.text(0.007, 7.5, 'y = ({0:6.2f})z + ({1:6.2f})\na = {0:6.2f} $\pm$ {2:6.2f}\nb = {1:6.2f} $\pm$ {3:4.2f}\n r = {4:6.4f}'.format(a, b, sigma_a, sigma_b, coeff_correlazione), c = 'r')
plt.legend()
plt.show()

Nei grafici successivi si utilizzano gli stessi valori per tracciare l'iperbole $V=\hat a/P+\hat b$ e quelle che delimitano la fascia di incertezza.

In [ ]:
plt.grid(which = 'both', color = '.85', linestyle = '-', linewidth=1)
plt.vlines(dati_pressione, dati_y-sigma, dati_y+sigma, linewidth = .5, color = 'b')
plt.fill_between(x_pres, y_presinf, y_pressup, alpha = .1, linewidth = 0, color='r')
plt.plot(x_pres, y_pres, color = 'red', linewidth = 2, label = 'iperbole di regressione')
plt.scatter(dati_pressione, dati_y, s = 20, c = 'cornflowerblue', zorder = 3, label = 'dati rilevati')
plt.title("Iperbole di regressione e fascia di incertezza")
plt.xlabel('pressione (kPa)')
plt.ylabel('volume (cm$^3$)')
plt.text(225, 15, 'y = ({0:6.2f})/x + ({1:6.2f})\na = {0:6.2f} $\pm$ {2:6.2f}\nb = {1:6.2f} $\pm$ {3:6.2f}\n r = {4:6.4f}'.format(a, b, sigma_a, sigma_b, coeff_correlazione), c = 'r')
plt.legend()
plt.show()

In [ ]:
plt.grid(which = 'both', color = '.85', linestyle = '-', linewidth = 1)
plt.xlim([min_pres-delta_pres, max_pres+delta_pres])
# la medesima scala del modello con intercetta nulla
plt.ylim([-.6,.6])
plt.vlines(dati_pressione, np.zeros(n), residui, linewidth = .5, color = 'orange')
plt.fill_between(x_pres, -sigma, sigma, alpha =.1, linewidth = 0, color = 'r')
plt.plot(x_pres, np.zeros(100), color = 'red', linewidth = 2)
plt.scatter(dati_pressione, residui, s = 20, c = 'cornflowerblue', zorder = 3, label = 'residui: $e_i=y_i-ax_i-b$')
plt.title("Distribuzione dei residui: modello $y=ax+b$\n devSt $\sigma=$" + str(round(sigma,2)))
plt.xlabel("pressione (kPa)")
plt.ylabel("residui (cm$^3$)")
plt.legend()
plt.show()